In [6]:
import pandas as pd
train_master_merge = pd.read_csv('train_master_merge.csv')
train_master_merge.head()

,Unnamed: 0,Idx,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,UserInfo_7,UserInfo_8,UserInfo_14,UserInfo_15,...,userinfo_IdAddress_num,userinfo_companySizeId_num,userinfo_HasPPDaiAccount_num,userinfo_PhoneType_num,userinfo_PPDaiAccount_num,userinfo_SecondEmail_num,userinfo_SecondMobile_num,userinfo_nickName_num,userinfo_HasSbOrGjj_num,userinfo_Position_num
0,0,10001,1.0,深圳,4.0,深圳,广东,深圳,2,2,...,0,0,0,0,0,0,0,0,0,0
1,1,10002,1.0,温州,4.0,温州,浙江,温州,3,3,...,0,0,0,0,0,0,0,0,0,0
2,2,10003,1.0,宜昌,3.0,宜昌,湖北,宜昌,4,4,...,0,0,0,0,0,0,0,0,0,0
3,3,10006,4.0,南平,1.0,南平,福建,南平,2,2,...,0,0,0,0,0,0,0,0,0,0
4,4,10007,5.0,辽阳,1.0,辽阳,辽宁,辽阳,3,3,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_master_merge['target']


0        0
1        0
2        0
3        0
4        0
        ..
28069    0
28070    0
28071    0
28072    0
28073    0
Name: target, Length: 28074, dtype: int64

In [9]:
X_train = train_master_merge.drop(['target'], axis=1)
y_train = train_master_merge['target']

In [10]:
#特征工程
import numpy as np

drop_columns = ['Idx','ListingInfo', 'UserInfo_20',  'UserInfo_19', 'UserInfo_8', 'UserInfo_7', 
                'UserInfo_4','UserInfo_2',
               'ListingInfo_timestamp', 'loginfo_last_day_timestamp', 'userinfo_last_day_timestamp']
X_train = X_train.drop(drop_columns, axis=1)
train_master_ = X_train
# 数值型特征（除去类型为object，除去'Idx', 'target'，除去binarized_features）
numerical_features = [f for f in train_master_.select_dtypes(exclude = ['object']).columns]

# 类别型特征（包括类型为object，除去'Idx', 'target'，除去binarized_features）
categorical_features = [f for f in train_master_.select_dtypes(include = ["object"]).columns]

feature_unique_count = []
# 遍历数值型特征，统计各个特征取值不为0的数量
for f in numerical_features:
    feature_unique_count.append((np.count_nonzero(train_master_[f].unique()), f))
    
# print(sorted(feature_unique_count))

# 遍历，将取值数量<=10的归为类别型特征
for c, f in feature_unique_count:
    if c <= 10:
        print('{} moved from numerical to categorical'.format(f))
        numerical_features.remove(f)
        categorical_features.append(f)

UserInfo_1 moved from numerical to categorical
UserInfo_3 moved from numerical to categorical
UserInfo_14 moved from numerical to categorical
UserInfo_15 moved from numerical to categorical
b_WeblogInfo_2 moved from numerical to categorical
b_WeblogInfo_5 moved from numerical to categorical
b_WeblogInfo_8 moved from numerical to categorical
b_WeblogInfo_9 moved from numerical to categorical
b_WeblogInfo_10 moved from numerical to categorical
b_WeblogInfo_11 moved from numerical to categorical
b_WeblogInfo_12 moved from numerical to categorical
b_WeblogInfo_13 moved from numerical to categorical
b_WeblogInfo_14 moved from numerical to categorical
b_WeblogInfo_15 moved from numerical to categorical
b_WeblogInfo_18 moved from numerical to categorical
b_UserInfo_5 moved from numerical to categorical
b_UserInfo_6 moved from numerical to categorical
b_UserInfo_9 moved from numerical to categorical
b_UserInfo_10 moved from numerical to categorical
b_UserInfo_16 moved from numerical to categor

In [11]:
X_train = pd.get_dummies(X_train, columns=categorical_features)

In [12]:
from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit_transform(X_train)

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# from scikitplot import plotters as skplt

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
#from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC

# 使用StratifiedKFold分层采样，保证预测target的分布合理，并且shuffle随机
cv = StratifiedKFold(n_splits=3, shuffle=True)

# 计算auc accuracy recall
def estimate(estimator, name='estimator'):
    auc = cross_val_score(estimator, X_train, y_train, scoring='roc_auc', cv=cv).mean()
    accuracy = cross_val_score(estimator, X_train, y_train, scoring='accuracy', cv=cv).mean()
    recall = cross_val_score(estimator, X_train, y_train, scoring='recall', cv=cv).mean()

    print("{}: auc:{:f}, recall:{:f}, accuracy:{:f}".format(name, auc, recall, accuracy))

In [15]:
#estimate(XGBClassifier(learning_rate=0.1, n_estimators=20, objective='binary:logistic'), 'XGBClassifier')
estimate(RidgeClassifier(), 'RidgeClassifier')
estimate(LogisticRegression(), 'LogisticRegression')
# estimate(RandomForestClassifier(), 'RandomForestClassifier')
estimate(AdaBoostClassifier(), 'AdaBoostClassifier')

RidgeClassifier: auc:0.736495, recall:0.000000, accuracy:0.944183


D:\conda\envs\py36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\conda\envs\py36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  ex

LogisticRegression: auc:0.743364, recall:0.010282, accuracy:0.942473
AdaBoostClassifier: auc:0.778263, recall:0.012853, accuracy:0.943720


In [19]:
from sklearn.ensemble import VotingClassifier

estimators = []
#estimators.append(('RidgeClassifier', RidgeClassifier()))
estimators.append(('LogisticRegression', LogisticRegression()))
#estimators.append(('XGBClassifier', XGBClassifier(learning_rate=0.1, n_estimators=20, objective='binary:logistic')))
estimators.append(('AdaBoostClassifier', AdaBoostClassifier()))
estimators.append(('RandomForestClassifier', RandomForestClassifier()))

#voting: auc:0.794587, recall:0.000642, accuracy:0.944433

voting = VotingClassifier(estimators = estimators, voting='soft')
estimate(voting, 'voting')

D:\conda\envs\py36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\conda\envs\py36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  ex

voting: auc:0.786445, recall:0.001929, accuracy:0.943934
